# Data preprocessing

How will you tokenize the data set? What will consist of a singular sequence, a poem, a stanza, or a line? Do you keep some words tokenized as bigrams? Do you split hyphenated words? How will you handle punctuation? 

Tokenize words in sonnet. Give HMM each line to train on.
Keep apostrophes and hyphens in a given word and remove other punctuation

Map words to a unique ID in a dictionary. Then map unique ID to number of syllables another dictionary.

In [1]:
import itertools
import nltk
from nltk.corpus import cmudict
nltk.download('cmudict')
import pickle
import networkx 
from networkx.algorithms.components.connected import connected_components

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\alyci\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [2]:
def get_sonnets(filename):
    """ Returns list of separate sonnets in the file. 
    Each line of a sonnet is a list of words (tokens). """
    with open(filename) as file:
        sons, son = [], []
        for line in file:
            line = line.strip()
            if line.isdigit() == False:
                lst = line.lower().translate(str.maketrans('', '', ':;,.?!()')).split()
                if len(lst) > 0: # count words in line
                    son.append(lst)
                elif len(son) > 0:
                    sons.append(son)
                    son = []
    sons.append(son) # add the final sonnet
    return sons

In [3]:
file = 'data/shakespeare.txt'
sonnets = get_sonnets(file)

# The sonnets 99 and 126 (at indices 98 and 125)
# do not count as sonnets, since they are not 14 lines long
for i in range(len(sonnets)):
    if len(sonnets[i]) != 14:
        print(i)
        
sonnets = sonnets[:98] + sonnets[99:125] + sonnets[126:]

98
125


In [4]:
d = cmudict.dict()
def count_syl(word):
    """ Returns number of syllables in a word. Taken
    from StackOverflow. """
    if word in d:
        return len(list(y for y in d[word][0] if y[-1].isdigit()))
    count = 0
    vowels = 'aeiouy'
    
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count

def get_syl(dic_to_ids):
    """ Returns dictionary of IDs to syllables given 
    a dictionary mapping words to ids. """
    dic = {}
    for word in dic_to_ids.keys():
        id_ = dic_to_ids[word]
        if "'" in word:
            dic[id_] = 1
        else:
            dic[id_] = sum(count_syl(w) for w in word.split("-"))
        
    return dic

In [5]:
def get_id(lines):
    """ Returns dictionary of words to IDs and 
    dictionary of IDs to words. """
    dic_to_ids, dic_to_words = {}, {}
    id_ = 0
    for line in lines:
        for word in line:
            if word not in dic_to_ids:
                dic_to_ids[word] = id_
                dic_to_words[id_] = word
                id_ += 1
    return dic_to_ids, dic_to_words

In [6]:
# Main function, which separates quatrains and couplets
# Set reverse to true to receive data with each line backwards
def get_mappings(sonnets, name):
    """ Returns dictionary of words to IDs, dictionary of IDs to words, 
    and dictionary of IDs to syllables. """    
    lines = list(itertools.chain.from_iterable(sonnets)) # list of lines
    dic_to_ids, dic_to_words = get_id(lines)
    dic_syl = get_syl(dic_to_ids)
    
    with open(name, 'wb') as f:
        pickle.dump((dic_to_ids, dic_to_words, dic_syl), f)
    
    return dic_to_ids, dic_to_words, dic_syl

In [7]:
dic_to_ids, dic_to_words, dic_syl = get_mappings(sonnets, 'shakespeare_dics.pkl')

# Additional Goals: Rhymes

In [8]:
def get_rhymes(sonnets):
    """ Returns list of pairs of rhyming words in each sonnet. """
    rhyme = []
    last_words = [] # compile all last words
    for son in sonnets:
        lst = [] # store list of last words in each line
        for line in son:
            lst.append(line[-1])
        last_words.append(lst)
    
    # get pairs of rhyming words
    for son in last_words:
        for i in [0, 1, 4, 5, 8, 9]:
            rhyme.append([son[i], son[i + 2]])
        rhyme.append([son[12], son[13]])
    
    # remove duplicates
    rhyme = [list(set(r)) for r in rhyme]
    return rhyme

In [9]:
def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # each sublist is a bunch of nodes
        G.add_nodes_from(part)
        # it also imlies a number of edges:
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

def get_all_rhymes(name):
    """ Returns list of lists of rhyming words from sonnets. """
    rhyme = get_rhymes(sonnets)
    G = to_graph(rhyme)
    all_r = connected_components(G)
    all_rhymes = [list(r) for r in all_r]
    
    with open(name, 'wb') as f:
        pickle.dump(all_rhymes, f)

    return all_rhymes

In [10]:
all_r = get_all_rhymes('shakespeare_rhymes.pkl')